<a href="https://colab.research.google.com/github/TheNizzo/SentimentAnalysis/blob/main/NLP_TP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
import random as rdn
import re

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset_train = load_dataset('imdb', split='train')
dataset_test = load_dataset('imdb', split='test')

In [31]:
x_train, y_train, x_test, y_test = dataset_train[:]['text'], dataset_train[:]['label'], dataset_test[:]['text'], dataset_test[:]['label']
len(x_train), len(x_test)

(25000, 25000)

#pretreatment

* Take out html

In [4]:
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

In [5]:
x_train_pretreated = [remove_tags(el) for el in x_train]
x_test_pretreated = [remove_tags(el) for el in x_test]

# Training

In [ ]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip
%cd fastText-0.9.2
!make
!pip install .

In [7]:
import fasttext

In [8]:
def write_list_to_file(filename, list_to_write):
  textfile = open(filename, "w")
  for element in list_to_write:
      textfile.write(element + "\n")
  textfile.close()

In [9]:
def fasttext_conversion(list_y, list_x):
  return ["__label__" + str(y) + " " + x for y, x in zip(list_y, list_x)]

In [41]:
def predict_many(x, y, n, predict_model, verbose=False):
  for i in range(n):
    r = rdn.randint(0,2500)
    print(r)
    print(predict_model.predict(x[r]), "| Actual value :", y[r])
    if verbose :
      print(x[r])

* No pretreament

In [10]:
train = fasttext_conversion(y_train,x_train)
test = fasttext_conversion(y_test,x_test)

In [11]:
rdn.shuffle(train)
rdn.shuffle(test)

In [12]:
write_list_to_file("train_file.txt", train)
write_list_to_file("test_file.txt", test)

In [13]:
model = fasttext.train_supervised(input="train_file.txt")

In [42]:
predict_many(x_test, y_test, 10, model)

673
(('__label__1',), array([0.67291683])) | Actual value : 1
1261
(('__label__1',), array([0.88899648])) | Actual value : 1
118
(('__label__0',), array([0.61611301])) | Actual value : 1
2261
(('__label__1',), array([0.94691557])) | Actual value : 1
2062
(('__label__1',), array([0.75161248])) | Actual value : 1
261
(('__label__1',), array([0.96387273])) | Actual value : 1
1546
(('__label__1',), array([0.79814684])) | Actual value : 1
1515
(('__label__1',), array([0.63852215])) | Actual value : 1
279
(('__label__1',), array([0.98535413])) | Actual value : 1
140
(('__label__1',), array([0.97815526])) | Actual value : 1


In [14]:
model.test("test_file.txt")

(25000, 0.85972, 0.85972)

* With pretreatment

In [15]:
train_pretreated = fasttext_conversion(y_train, x_train_pretreated)
test_pretreated = fasttext_conversion(y_test, x_test_pretreated)

In [16]:
rdn.shuffle(train_pretreated)
rdn.shuffle(test_pretreated)

In [17]:
write_list_to_file("train_file_pretreated.txt", train_pretreated)
write_list_to_file("test_file_pretreated.txt", test_pretreated)

In [39]:
model_pretreated = fasttext.train_supervised(input="train_file_pretreated.txt")

In [46]:
predict_many(x_test_pretreated, y_test, 10, model_pretreated, True)

1099
(('__label__1',), array([0.99795216])) | Actual value : 1
Being from the Buffalo area I was well aware of the movie having read many articles in local publications. I was most impressed with the movie, especially its clever plot, the acting and the local scenes. Nice to see so many older, quality stars in the various roles. I feel that especially those of us over 50 will find the movie excellent and you can leave the theater feeling that your time was well spent.
1558
(('__label__1',), array([0.85770649])) | Actual value : 1
Boogie Nights is perhaps one of the greatest examples any would-be filmmaker should take a long hard look at. Sure, you could spend loads of quality time reviewing the clasics from Hitchcock to Scorsese; but lets follow suit for the modern generation and study half-heartedly.Where to begin, I suppose one could look at the film as simply a story, perhaps even docudrama which focuses on the late 1970's porn industry-and what an industry it was! The other half co

In [19]:
model_pretreated("test_file_pretreated.txt")

(25000, 0.86072, 0.86072)